In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from seaborn import set_style
import kagglehub
set_style("whitegrid")

In [2]:
path = kagglehub.dataset_download("ehallmar/daily-historical-stock-prices-1970-2018")

print("Path to dataset files:", path)

Path to dataset files: /home/hava/.cache/kagglehub/datasets/ehallmar/daily-historical-stock-prices-1970-2018/versions/1


In [3]:
prices_file = path + '/historical_stock_prices.csv'
labels_file = path + '/historical_stocks.csv'
df_prices = pd.read_csv(prices_file)
df_labels = pd.read_csv(labels_file)
df_prices

,ticker,open,close,adj_close,low,high,volume,date
0,AHH,11.50,11.58,8.493155,11.25,11.68,4633900,2013-05-08
1,AHH,11.66,11.55,8.471151,11.50,11.66,275800,2013-05-09
2,AHH,11.55,11.60,8.507822,11.50,11.60,277100,2013-05-10
3,AHH,11.63,11.65,8.544494,11.55,11.65,147400,2013-05-13
4,AHH,11.60,11.53,8.456484,11.50,11.60,184100,2013-05-14
...,...,...,...,...,...,...,...,...
20973884,NZF,14.60,14.59,14.590000,14.58,14.62,137500,2018-08-20
20973885,NZF,14.60,14.58,14.580000,14.57,14.61,151200,2018-08-21
20973886,NZF,14.58,14.59,14.590000,14.57,14.63,185400,2018-08-22
20973887,NZF,14.60,14.57,14.570000,14.57,14.64,135600,2018-08-23


In [4]:
sector = df_labels[df_labels['sector']=='TRANSPORTATION'][['ticker','name']]
sector['ticker']=sector['ticker'].astype(str)
print(sector['ticker'])
df_sector = df_prices[df_prices['ticker'].isin(sector['ticker'])].reset_index()
df_sector

86      AIRT
87      ATSG
110     ALGT
150      AAL
217     ARCB
        ... 
6209     UNP
6212     UAL
6214     UPS
6239    USDP
6457     ZTO
Name: ticker, Length: 120, dtype: object


,index,ticker,open,close,adj_close,low,high,volume,date
0,222206,CAAP,16.932270,16.334661,16.334661,16.314741,17.918327,10433400,2018-02-01
1,222214,CAAP,16.270000,16.600000,16.600000,16.260000,16.950001,1527100,2018-02-02
2,222228,CAAP,16.600000,16.490000,16.490000,16.420000,16.879999,1110200,2018-02-05
3,222238,CAAP,16.500000,16.500000,16.500000,16.410000,16.700001,822700,2018-02-06
4,222251,CAAP,16.490000,16.490000,16.490000,16.320000,16.639999,473900,2018-02-07
...,...,...,...,...,...,...,...,...,...
418332,20870703,HA,43.000000,43.950001,43.950001,42.799999,44.200001,601500,2018-08-20
418333,20870705,HA,43.950001,43.700001,43.700001,43.500000,44.250000,418800,2018-08-21
418334,20870707,HA,43.599998,41.950001,41.950001,41.700001,43.599998,591500,2018-08-22
418335,20870709,HA,41.849998,41.200001,41.200001,40.849998,42.230000,585300,2018-08-23


In [5]:
df_sector['date'] = pd.to_datetime(df_sector['date'])
df_sector['ticker'] = (df_sector['ticker']).astype(str)
dst_dates_df = pd.read_csv('DaylightSavingsTimeChangeDates_1971-2024.csv')
dst_dates_spring = pd.to_datetime(dst_dates_df.Date+" "+dst_dates_df.Year.astype(str))
dst_dates_fall = pd.to_datetime(dst_dates_df['Date.1']+" "+dst_dates_df.Year.astype(str))
dst_dates = pd.concat([dst_dates_fall,dst_dates_spring])
dst_mon = dst_dates+pd.DateOffset(1)
dst_fri = dst_dates+pd.DateOffset(-2)
dst_fall_mon = dst_dates_fall+pd.DateOffset(1)
dst_fall_fri = dst_dates_fall+pd.DateOffset(-2)
dst_spring_mon = dst_dates_spring+pd.DateOffset(1)
dst_spring_fri = dst_dates_spring+pd.DateOffset(-2)

In [6]:
sector_dst_mon_fall = df_sector[df_sector.date.isin(dst_fall_mon)]
sector_dst_mon_fall = sector_dst_mon_fall.assign(Season='Fall')
sector_dst_mon_spring = df_sector[df_sector.date.isin(dst_spring_mon)]
sector_dst_mon_spring = sector_dst_mon_spring.assign(Season='Spring')
sector_dst_mon = pd.concat([sector_dst_mon_fall,sector_dst_mon_spring])
sector_dst_fri = df_sector[df_sector.date.isin(dst_fri)]

for tkr in sector['ticker'].astype(str):
    mon_tkr = sector_dst_mon[sector_dst_mon.ticker.astype(str)==tkr]
    fri_tkr = sector_dst_fri[sector_dst_fri.ticker.astype(str)==tkr]
    sector_dst_mon = sector_dst_mon.drop(mon_tkr[mon_tkr.date.isin(fri_tkr.date+pd.DateOffset(3))==False].index)
    sector_dst_fri = sector_dst_fri.drop(fri_tkr[fri_tkr.date.isin(mon_tkr.date+pd.DateOffset(-3))==False].index)
    #finance_dst_mon = finance_dst_mon.drop(finance_dst_mon[(finance_dst_mon.ticker.astype(str)==tkr  finance_dst_mon['date'].isin(finance_dst_fri[finance_dst_fri.ticker.astype(str)==tkr].date+pd.DateOffset(3))==False)].index)
    #finance_dst_fri = finance_dst_fri.drop(finance_dst_fri[(finance_dst_fri['ticker'==tkr] & finance_dst_fri['date'].isin(finance_dst_mon['ticker'==tkr].date+pd.DateOffset(-3))==False)].index)
print(len(sector_dst_mon))
print(len(sector_dst_fri))

3156
3156


In [7]:
sector_dst_fri = sector_dst_fri.reset_index()
sector_dst_mon = sector_dst_mon.reset_index()

In [8]:
sector_dst = pd.DataFrame()
sector_dst['ticker'] = sector_dst_mon['ticker']
sector_dst['mon_date'] = sector_dst_mon['date']
sector_dst['mon_open'] = sector_dst_mon['open']
sector_dst['mon_close'] = sector_dst_mon['close']
sector_dst['mon_adj_close'] = sector_dst_mon['adj_close']
sector_dst['mon_low'] = sector_dst_mon['low']
sector_dst['mon_high'] = sector_dst_mon['high']
sector_dst['mon_volume'] = sector_dst_mon['volume']
sector_dst['fri_date'] = sector_dst_fri['date']
sector_dst['fri_open'] = sector_dst_fri['open']
sector_dst['fri_close'] = sector_dst_fri['close']
sector_dst['fri_adj_close'] = sector_dst_fri['adj_close']
sector_dst['fri_low'] = sector_dst_fri['low']
sector_dst['fri_high'] = sector_dst_fri['high']
sector_dst['fri_volume'] = sector_dst_fri['volume']
sector_dst['Season'] = sector_dst_mon['Season']
sector_dst

,ticker,mon_date,mon_open,mon_close,mon_adj_close,mon_low,mon_high,mon_volume,fri_date,fri_open,fri_close,fri_adj_close,fri_low,fri_high,fri_volume,Season
0,SSWN,2014-11-03,25.500000,25.549999,21.120195,25.500000,25.549999,2800,2018-03-09,13.770000,14.420000,14.420000,13.700000,14.570000,1748800,Fall
1,SSWN,2015-11-02,24.899000,24.860001,20.882948,24.860001,24.899000,1100,2014-10-31,25.740000,25.500000,21.078861,25.500000,25.750000,18100,Fall
2,SSWN,2016-11-07,24.809999,24.760000,22.182564,24.610001,24.950001,27000,2015-03-06,25.780001,25.750000,21.630566,25.750000,25.893999,3700,Fall
3,SSWN,2017-11-06,25.381001,25.438999,24.271755,25.381001,25.490000,8100,2015-10-30,24.868000,24.830000,20.857746,24.830000,24.900000,9500,Fall
4,TNP,2002-10-28,5.950000,5.965000,2.088306,5.950000,5.970000,3200,2016-03-11,24.540001,24.350000,20.801964,24.350000,24.680000,51700,Fall
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3151,HA,2014-03-10,14.200000,14.550000,14.369158,14.050000,14.590000,1247300,2016-03-11,44.860001,45.349998,44.786343,44.860001,45.880001,862900,Spring
3152,HA,2015-03-09,19.020000,19.200001,18.961363,18.930000,19.410000,1443700,2016-11-04,44.500000,45.500000,44.934479,44.279999,46.099998,577800,Spring
3153,HA,2016-03-14,44.860001,45.750000,45.181370,44.840000,46.070000,1010400,2017-03-10,48.799999,48.200001,47.600922,47.790001,49.000000,849900,Spring
3154,HA,2017-03-13,48.000000,48.799999,48.193459,47.750000,49.230000,696000,2017-11-03,32.900002,33.500000,33.083626,32.599998,33.610001,1095800,Spring


In [9]:
sector_mon = df_sector[df_sector.date.dt.day_name()=='Monday']
sector_mon = sector_mon.drop(sector_mon[sector_mon.date.isin(dst_mon)].index)
sector_fri = df_sector[df_sector.date.dt.day_name()=='Friday']
sector_fri = sector_fri.drop(sector_fri[sector_fri.date.isin(dst_fri)].index)
for tkr in sector['ticker'].astype(str):
    mon_tkr = sector_mon[sector_mon.ticker.astype(str)==tkr]
    fri_tkr = sector_fri[sector_fri.ticker.astype(str)==tkr]
    sector_mon = sector_mon.drop(mon_tkr[mon_tkr.date.isin(fri_tkr.date+pd.DateOffset(3))==False].index)
    sector_fri = sector_fri.drop(fri_tkr[fri_tkr.date.isin(mon_tkr.date+pd.DateOffset(-3))==False].index)
print(len(sector_mon))
print(len(sector_fri))

71242
71242


In [10]:
sector_mon = sector_mon.reset_index()
sector_fri = sector_fri.reset_index()

In [11]:
sector_wkd = pd.DataFrame()
sector_wkd['ticker'] = sector_mon['ticker']
sector_wkd['mon_date'] = sector_mon['date']
sector_wkd['mon_open'] = sector_mon['open']
sector_wkd['mon_close'] = sector_mon['close']
sector_wkd['mon_adj_close'] = sector_mon['adj_close']
sector_wkd['mon_low'] = sector_mon['low']
sector_wkd['mon_high'] = sector_mon['high']
sector_wkd['mon_volume'] = sector_mon['volume']
sector_wkd['fri_date'] = sector_fri['date']
sector_wkd['fri_open'] = sector_fri['open']
sector_wkd['fri_close'] = sector_fri['close']
sector_wkd['fri_adj_close'] = sector_fri['adj_close']
sector_wkd['fri_low'] = sector_fri['low']
sector_wkd['fri_high'] = sector_fri['high']
sector_wkd['fri_volume'] = sector_fri['volume']
sector_wkd

,ticker,mon_date,mon_open,mon_close,mon_adj_close,mon_low,mon_high,mon_volume,fri_date,fri_open,fri_close,fri_adj_close,fri_low,fri_high,fri_volume
0,CAAP,2018-02-05,16.600000,16.490000,16.490000,16.420000,16.879999,1110200,2018-02-02,16.270000,16.600000,16.600000,16.260000,16.950001,1527100
1,CAAP,2018-02-12,16.000000,16.150000,16.150000,15.920000,16.190001,94300,2018-02-09,15.820000,16.000000,16.000000,15.160000,16.090000,454100
2,CAAP,2018-02-26,15.580000,15.820000,15.820000,15.530000,15.890000,124400,2018-02-23,15.790000,15.550000,15.550000,15.350000,15.940000,123700
3,CAAP,2018-03-05,15.310000,14.960000,14.960000,14.960000,15.310000,83300,2018-03-02,15.630000,15.300000,15.300000,15.120000,15.630000,38600
4,CAAP,2018-03-19,14.150000,13.770000,13.770000,13.650000,14.380000,154700,2018-03-16,14.090000,14.140000,14.140000,14.000000,14.490000,151500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71237,HA,2018-07-23,37.400002,37.900002,37.793491,36.750000,38.099998,703800,2018-07-20,37.700001,37.500000,37.394615,37.150002,37.950001,777400
71238,HA,2018-07-30,39.450001,39.799999,39.688148,39.099998,40.049999,684800,2018-07-27,39.849998,39.299999,39.189556,39.000000,40.799999,818400
71239,HA,2018-08-06,41.250000,41.200001,41.084217,40.849998,41.750000,480700,2018-08-03,39.799999,41.150002,41.034359,39.799999,41.599998,610000
71240,HA,2018-08-13,42.150002,41.900002,41.782249,41.299999,42.150002,335100,2018-08-10,41.700001,42.000000,41.881966,41.200001,42.150002,361000


In [12]:
sector_dst.to_csv('DST_Transport_Stocks.csv')
sector_wkd.to_csv('Weekends_Transport_Stocks.csv')